<a href="https://colab.research.google.com/github/NSuprotivniy/anti-spoofing/blob/master/nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-o3jwt63t
  Stored in directory: /tmp/pip-ephem-wheel-cache-bpahkvtt/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [2]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, Dropout
from keras_vggface.vggface import VGGFace
from keras.utils import Sequence
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [0]:
import os
from glob import glob
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [0]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [0]:
def get_model():
  vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3), model='resnet50')
  vgg_model.trainable = False
  last_layer = vgg_model.get_layer('avg_pool').output
  x = Flatten(name='flatten')(last_layer)
  x = Dense(2048, activation='relu', name='dense_relu')(x)
  x = Dropout(0.5, seed=17)(x)
  out = Dense(1, activation='sigmoid', name='classifier')(x)
  custom_vgg_model = Model(vgg_model.input, out)
  custom_vgg_model.compile(optimizer=Adam(lr=0.000001), loss='binary_crossentropy', metrics=[auc])
  return custom_vgg_model

In [6]:
drive.mount("/content/nsuprotivniy/")
data_path = "/content/nsuprotivniy/My Drive/Colab Notebooks/data/LCC_FASD/LCC_FASD_development"

Drive already mounted at /content/nsuprotivniy/; to attempt to forcibly remount, call drive.mount("/content/nsuprotivniy/", force_remount=True).


In [0]:
class KfoldGenerator():  
  def __init__(self, batch_size=32, folds=10, split=0.1, seed=17):
    spoof_path = glob(os.path.join(data_path, "spoof/cropped/*.npy"))
    real_path = glob(os.path.join(data_path, "real/cropped/*.npy"))
    X = np.concatenate((spoof_path, real_path))
    y = np.concatenate((np.ones(len(spoof_path)), np.zeros(len(real_path))))
    
    np.random.seed(seed)
    ids = np.random.permutation(len(X))
    
    if len(X) % batch_size != 0:
      to_add = np.random.choice(ids, -len(X) % batch_size)
      ids = np.concatenate((ids, to_add))
      
    X, y = X[ids], y[ids]
    
    X_batches = np.array(np.split(X, len(X) // batch_size))
    y_batches = np.array(np.split(y, len(X) // batch_size))
      
    X_train, X_test, y_train, y_test = train_test_split(X_batches, y_batches, 
                                                        test_size=split, 
                                                        random_state=seed)
    self.test = X_test, y_test
    self.train = X_train, y_train
    
    kfold = KFold(n_splits=folds, random_state=seed)
    self.train_folds = [(X_batches[train], y_batches[train], 
                         X_batches[test], y_batches[test]) 
                        for train, test in kfold.split(X_batches, y_batches)]

  def get_test(self):
    return self.test
  
  def get_train(self):
    return self.train
  
  def get_train_folds(self):
    return self.train_folds

In [0]:
class Generator(Sequence):  
  def __init__(self, X_batches, y_batches):
    self.X_batches = X_batches
    self.y_batches = y_batches
    
  def __len__(self):
    return len(self.X_batches)

  def __getitem__(self, idx):
    X = np.array([np.load(path) for path in self.X_batches[idx]])
    y = self.y_batches[idx]
    return X, y

In [0]:
kfoldgenrator = KfoldGenerator()

In [0]:
metrics_path = os.path.join(data_path, "metrics")
if (not os.path.exists(metrics_path)):
  os.mkdir(metrics_path)
def save_metric(metrics, i):
  with open(os.path.join(metrics_path, str(i) + "_metrics"), "w") as f:
    f.write("fold {}: loss {}, auc {}".format(i, metrics[0], metrics[1]))
    
def save_metric(metrics):
  with open(os.path.join(metrics_path, "metrics"), "w") as f:
    f.write("loss {}, auc {}".format(i, metrics[0], metrics[1]))

In [0]:
checkpoints_path = os.path.join(data_path, "checkpoints")
if (not os.path.exists(checkpoints_path)):
  os.mkdir(checkpoints_path)

In [0]:
for i, (X_train, y_train, X_test, y_test) in enumerate(kfoldgenrator.get_train_folds()):
  checkpoint_filename = "fold_" + str(i) + "_weights.h5"
  checkpoint = [ModelCheckpoint(os.path.join(checkpoints_path, checkpoint_filename), verbose=1, period=5)]
  train_gen = Generator(X_train, y_train)
  model = get_model()
  model.fit_generator(train_gen, epochs=5, verbose=1, callbacks=checkpoint)
  test_gen = Generator(X_test, y_test)
  metrics = model.evaluate_generator(test_gen, verbose=1)
  print("fold {}: loss {}, auc {}".format(i, metrics[0], metrics[1]))
  save_metric(metrics, i)

In [27]:
X_train, y_train = kfoldgenrator.get_train()
X_test, y_test = kfoldgenrator.get_test()
checkpoint_filename = "final_model_weights.h5"
checkpoint = [ModelCheckpoint(os.path.join(checkpoints_path, checkpoint_filename), verbose=1, period=5)]
train_gen = Generator(X_train, y_train)
model = get_model()
model.load_weights(os.path.join(checkpoints_path, checkpoint_filename))
model.fit_generator(train_gen, epochs=10, verbose=1, callbacks=checkpoint)
test_gen = Generator(X_test, y_test)
metrics = model.evaluate_generator(test_gen, verbose=1)
print("loss {}, auc {}".format(metrics[0], metrics[1]))
save_metric(metrics)

Epoch 1/10
83/83 [==============================] - 101s 1s/step - loss: 0.0663 - auc: 0.9941
Epoch 2/10
83/83 [==============================] - 34s 409ms/step - loss: 0.0483 - auc: 0.9955
Epoch 3/10
83/83 [==============================] - 34s 411ms/step - loss: 0.0425 - auc: 0.9967
Epoch 4/10
83/83 [==============================] - 34s 413ms/step - loss: 0.0302 - auc: 0.9974
Epoch 5/10
83/83 [==============================] - 34s 411ms/step - loss: 0.0297 - auc: 0.9978

Epoch 00005: saving model to /content/nsuprotivniy/My Drive/Colab Notebooks/data/LCC_FASD/LCC_FASD_development/checkpoints/final_model_weights.h5
Epoch 6/10
83/83 [==============================] - 34s 412ms/step - loss: 0.0224 - auc: 0.9980
Epoch 7/10
83/83 [==============================] - 34s 415ms/step - loss: 0.0180 - auc: 0.9983
Epoch 8/10
83/83 [==============================] - 34s 410ms/step - loss: 0.0156 - auc: 0.9986
Epoch 9/10
83/83 [==============================] - 34s 412ms/step - loss: 0.0118 - auc